# MonadIOControl
###### (https://www.fpcomplete.com/blog/2017/06/tale-of-two-brackets)

Let's say we want to implement the error handling family of functions specifically for the `StateT` monad transformer

In [62]:
:m Control.Monad.Trans.State
:m Control.Exception

tryStateT :: Exception e => StateT s IO a -> StateT s IO (Either e a)
tryStateT (StateT f) = StateT $ \s0 -> do
  eres <- try $ f s0
  case eres of
    Left err -> pure (Left err, s0)
    Right (a, s1) -> pure (Right a, s1)
    
catchStateT :: Exception e => StateT s IO a -> (e -> StateT s IO a) -> StateT s IO a
catchStateT (StateT f) errHandler = StateT $ \s0 -> do
  catch (f s0) $ \err ->
    runStateT (errHandler err) s0
    
finallyStateT :: StateT s IO a -> StateT s IO b -> StateT s IO a
finallyStateT (StateT f) (StateT final) = StateT $ \s0 -> mask $ \restore -> do
  (a, s1) <- restore (f s0) `onException` final s0
  (_b, s2) <- final s1
  pure (a, s2)

Note that other possible implementations exist for `finallyStateT` which do not propogate the state fully. The final action could be a plain IO, or the initial state could be used for both the action and the final. Having a plain IO final will have better performance characteristics because it doesn't have to deal with the extra state.

Now we can try to generalize this approach for any MonadTransformer. Let's start by factoring out the parts that deal with StateT specifically from the parts that don't

In [61]:
:ext RankNTypes

type Run s = forall b. StateT s IO b -> IO (b, s)

capture :: forall s a. (Run s -> IO a) -> StateT s IO a
capture withRun = StateT $ \s -> do
  let run :: Run s
      run (StateT f) = f s
  a <- withRun run
  pure (a, s)
  
restoreState :: (a, s) -> StateT s IO a
restoreState st = StateT $ \_s -> pure st

finallyStateT :: StateT s IO a -> StateT s IO b -> StateT s IO a
finallyStateT action final = do
  x <- capture $ \run -> run action `onException` run final
  a <- restoreState x
  _b <- final
  pure a

This confused me until I realized that the `a` in `capture` is typically going to be `(a, s)`.

Now let's write a type class that generalizes this construction over the IO monad.

In [43]:
:ext TypeFamilies
:m Control.Monad.IO.Class

type RunInIO m = forall b. MonadIOControl m => m b -> IO (StM m b)

class MonadIO m => MonadIOControl m where
  type StM m a :: *
  
  liftIOWith :: (RunInIO m -> IO a) -> m a
  restoreM :: StM m a -> m a 

`listIOWith` takes a function which tells it how to take some function of the form `RunInIO` (which takes an action and produces the state context in IO) and produce an IO result from it. Given such a function, it produces a stateful computation of the same result type. So by passing the `withRun` argument function a `RunInIO` we get an IO action whose result we need to then represent in our monadic context.

Writing an `IO` instance of this class is easy

In [44]:
instance MonadIOControl IO where
  type StM IO a = a -- there's no additional state
  
  liftIOWith withRun = withRun id -- no additional context
  restoreM = pure

Now let's write a `StateT` instance

In [51]:
{-
instance MonadIOControl (StateT s IO) where
  type StM (StateT s IO) a = (a, s)
  
  liftIOWith withRun = StateT $ \s -> do 
    a <- withRun $ \(StateT f) -> f s
    pure (a, s)
  restoreM as = StateT $ \_s -> pure as
-}

instance MonadIOControl m => MonadIOControl (StateT s m) where
  type StM (StateT s m) a = StM m (a, s)
  
  liftIOWith withRun = StateT $ \s -> do
    a <- liftIOWith $ \mbToIOStMmb ->
      withRun (\(StateT f) -> mbToIOStMmb $ f s)
    pure (a, s)
    
  restoreM as = StateT $ \_s -> restoreM as

`MonadIOControl` can be generalized to `MonadBaseControl`